# Simulation 07 -- Defense Layer Scorecard

## Purpose
Run **all 5 defense layers** against **6 stress scenarios** and produce a comprehensive scorecard.

**Key question:** Which defense layers are reliable, which are redundant, and which have blind spots?

## Defense Layers
1. **Isolation Forest** -- Independent anomaly detection agreement (Cohen's Kappa)
2. **Local Outlier Factor (LOF)** -- Density-based anomaly agreement
3. **External Benchmark Compliance** -- Predicted FMV within industry rent ranges
4. **Land Value Correlation** -- Predicted FMV correlates with underlying land value
5. **(Simulated) Domain Expert** -- Agreement with expert estimates

## Stress Scenarios
- A: Baseline (clean data)
- B: Symmetric noise 20%
- C: Symmetric noise 50%
- D: Asymmetric bias 20%
- E: Asymmetric bias 40%
- F: Missing key features

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import median_abs_deviation, pearsonr, spearmanr
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import (mean_absolute_error, r2_score, silhouette_score,
                             cohen_kappa_score, confusion_matrix, classification_report)
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

try:
    from hdbscan import HDBSCAN
except ImportError:
    from sklearn.cluster import HDBSCAN

try:
    from quantile_forest import RandomForestQuantileRegressor
    HAS_QRF = True
except ImportError:
    HAS_QRF = False
    print('quantile_forest not installed -- using RF proxy for quantiles')

np.random.seed(42)
print('Imports ready.')

## 1. Load Data & Define Feature Lists

In [ ]:
df_raw = pd.read_parquet('../data/synthetic/synthetic_5k_sites.parquet')
print(f'Loaded {len(df_raw)} sites, {len(df_raw.columns)} columns')

X_MARKET = [
    'census_population_density', 'census_population_3km', 'census_median_income',
    'census_median_home_value', 'census_median_rent', 'census_vacancy_rate',
    'urban_rural_class', 'underlying_land_value_psf', 'hud_safmr',
    'property_tax_rate', 'topography_class', 'terrain_elevation_variance_m',
    'poi_density_1km', 'building_density_500m', 'distance_to_highway_km',
    'ground_elevation_ft', 'unemployment_rate_local',
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm',
    'competitor_tower_distance_km', 'tower_per_capita',
    'scarcity_index', 'landlord_concentration', 'composite_hazard_score',
    'tower_height_ft', 'lease_area_sqft', 'parcel_acreage', 'construction_age',
    'site_access_ordinal', 'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft',
    'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
]

Z_NETWORK = [
    'tenant_count', 'primary_technology', 'coverage_critical',
    'capacity_utilization_pct', 'high_traffic', 'backhaul_type',
    'has_fiber_connection', 'network_substitutability_score',
    'search_ring_alternative_count', 'inter_site_distance_km',
    'antenna_count', 'structure_capacity_remaining', 'site_priority_tier',
]

ALL_FEATURES = X_MARKET + Z_NETWORK

In [ ]:
def encode_categoricals(df):
    """Encode categoricals and return the encoded feature columns."""
    dfc = df.copy()
    le_struct = LabelEncoder()
    le_zone = LabelEncoder()
    dfc['structure_type_enc'] = le_struct.fit_transform(dfc['structure_type'].astype(str))
    dfc['zoning_district_type_enc'] = le_zone.fit_transform(dfc['zoning_district_type'].astype(str))
    prop_dummies = pd.get_dummies(dfc['property_type'], prefix='pt')
    dfc = pd.concat([dfc, prop_dummies], axis=1)
    cat_cols = ['structure_type_enc', 'zoning_district_type_enc'] + list(prop_dummies.columns)
    return dfc, cat_cols

df_enc, CAT_COLS = encode_categoricals(df_raw)
X_MARKET_ENC = X_MARKET + CAT_COLS
ALL_FEATURES_ENC = ALL_FEATURES + CAT_COLS
print(f'X_MARKET_ENC: {len(X_MARKET_ENC)} features')
print(f'ALL_FEATURES_ENC: {len(ALL_FEATURES_ENC)} features')

## 2. Reusable Pipeline Function

In [ ]:
def run_full_pipeline(df, x_market_enc=None, all_features_enc=None):
    """Run the complete FMV estimation pipeline.
    Returns dict with predictions and metrics."""
    if x_market_enc is None:
        x_market_enc = X_MARKET_ENC
    if all_features_enc is None:
        all_features_enc = ALL_FEATURES_ENC
    
    result = {}
    
    # Stage 1: Clustering
    X_clust = StandardScaler().fit_transform(df[x_market_enc].values)
    clusterer = HDBSCAN(min_cluster_size=20, min_samples=5)
    df = df.copy()
    df['cluster_id'] = clusterer.fit_predict(X_clust)
    
    # Stage 2: Pseudo-labeling
    df['pseudo_label'] = 'MARGINAL'
    for cid in df['cluster_id'].unique():
        if cid == -1:
            continue
        mask = df['cluster_id'] == cid
        rents = df.loc[mask, 'current_monthly_rent']
        med = rents.median()
        mad = median_abs_deviation(rents, scale='normal')
        if mad == 0:
            mad = rents.std() or 1.0
        z = np.abs(rents - med) / mad
        df.loc[mask & (z < 1.0), 'pseudo_label'] = 'CORE'
        df.loc[mask & (z >= 2.0), 'pseudo_label'] = 'OUTLIER'
    
    core_mask = df['pseudo_label'] == 'CORE'
    result['core_pct'] = core_mask.sum() / len(df)
    
    # Stage 3: Supervised model
    available_feats = [f for f in all_features_enc if f in df.columns]
    X_core = df.loc[core_mask, available_feats].values.astype(float)
    y_core = df.loc[core_mask, 'current_monthly_rent'].values
    groups = df.loc[core_mask, 'state'].values
    
    # XGBoost with spatial CV
    xgb = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=6,
                       subsample=0.8, colsample_bytree=0.8, random_state=42, verbosity=0)
    splitter = GroupKFold(n_splits=min(5, len(np.unique(groups))))
    oof = np.zeros(len(y_core))
    for tr_idx, va_idx in splitter.split(X_core, y_core, groups):
        xgb.fit(X_core[tr_idx], y_core[tr_idx])
        oof[va_idx] = xgb.predict(X_core[va_idx])
    xgb.fit(X_core, y_core)
    
    # QRF
    X_all = df[available_feats].values.astype(float)
    if HAS_QRF:
        qrf = RandomForestQuantileRegressor(n_estimators=300, min_samples_leaf=5, random_state=42, n_jobs=-1)
        qrf.fit(X_core, y_core)
        q_preds = qrf.predict(X_all, quantiles=[0.25, 0.50, 0.75])
        qrf_q25 = q_preds[:, 0]
        qrf_q75 = q_preds[:, 2]
    else:
        rf = RandomForestRegressor(n_estimators=300, min_samples_leaf=5, random_state=42, n_jobs=-1)
        rf.fit(X_core, y_core)
        tree_preds = np.array([t.predict(X_all) for t in rf.estimators_])
        qrf_q25 = np.percentile(tree_preds, 25, axis=0)
        qrf_q75 = np.percentile(tree_preds, 75, axis=0)
    
    # Meta-learner
    meta = Ridge(alpha=1.0)
    meta.fit(oof.reshape(-1, 1), y_core)
    
    # Stage 4: Predict and classify
    xgb_pred = xgb.predict(X_all)
    predicted_fmv = meta.predict(xgb_pred.reshape(-1, 1))
    
    model_class = np.where(df['current_monthly_rent'].values > qrf_q75, 'OVERPAYING',
                  np.where(df['current_monthly_rent'].values < qrf_q25, 'UNDERPAYING', 'AT_MARKET'))
    
    # Metrics vs ground truth
    true_fmv = df['true_fmv'].values
    result['mape'] = np.mean(np.abs(predicted_fmv - true_fmv) / true_fmv)
    result['mae'] = mean_absolute_error(true_fmv, predicted_fmv)
    result['r2'] = r2_score(true_fmv, predicted_fmv)
    result['bias'] = np.mean(predicted_fmv - true_fmv)
    
    true_class = df['true_class'].values
    result['class_accuracy'] = np.mean(model_class == true_class)
    
    result['predicted_fmv'] = predicted_fmv
    result['model_class'] = model_class
    result['qrf_q25'] = qrf_q25
    result['qrf_q75'] = qrf_q75
    result['df'] = df
    
    return result

## 3. Define Defense Layer Functions

In [ ]:
def defense_layer_1_isolation_forest(df, predicted_fmv, model_class):
    """Layer 1: Isolation Forest anomaly detection agreement."""
    feats = [f for f in ALL_FEATURES if f in df.columns]
    X = df[feats].values.astype(float)
    rent = df['current_monthly_rent'].values.reshape(-1, 1)
    X_with_rent = np.hstack([StandardScaler().fit_transform(X),
                              StandardScaler().fit_transform(rent)])
    
    iso = IsolationForest(n_estimators=500, contamination=0.15, random_state=42)
    iso_labels = iso.fit_predict(X_with_rent)  # 1=normal, -1=anomaly
    
    # Assign direction to anomalies
    if_class = np.full(len(df), 'AT_MARKET')
    anomaly = iso_labels == -1
    if_class[anomaly & (df['current_monthly_rent'].values > predicted_fmv)] = 'OVERPAYING'
    if_class[anomaly & (df['current_monthly_rent'].values < predicted_fmv)] = 'UNDERPAYING'
    
    kappa = cohen_kappa_score(model_class, if_class)
    agreement = np.mean(model_class == if_class)
    return {'kappa': kappa, 'agreement': agreement, 'if_class': if_class}


def defense_layer_2_lof(df, predicted_fmv, model_class):
    """Layer 2: Local Outlier Factor agreement."""
    feats = [f for f in ALL_FEATURES if f in df.columns]
    X = df[feats].values.astype(float)
    rent = df['current_monthly_rent'].values.reshape(-1, 1)
    X_with_rent = np.hstack([StandardScaler().fit_transform(X),
                              StandardScaler().fit_transform(rent)])
    
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.15)
    lof_labels = lof.fit_predict(X_with_rent)  # 1=inlier, -1=outlier
    
    lof_class = np.full(len(df), 'AT_MARKET')
    outlier = lof_labels == -1
    lof_class[outlier & (df['current_monthly_rent'].values > predicted_fmv)] = 'OVERPAYING'
    lof_class[outlier & (df['current_monthly_rent'].values < predicted_fmv)] = 'UNDERPAYING'
    
    kappa = cohen_kappa_score(model_class, lof_class)
    agreement = np.mean(model_class == lof_class)
    return {'kappa': kappa, 'agreement': agreement, 'lof_class': lof_class}


def defense_layer_3_benchmarks(df, predicted_fmv):
    """Layer 3: External benchmark compliance."""
    urban_class = df['urban_rural_class'].values
    is_rooftop = (df['property_type'].values == 'rooftop').astype(int)
    
    # Define benchmark ranges by urban class
    bench_low = np.where(urban_class == 4, 1200,
                np.where(urban_class == 3, 800,
                np.where(urban_class == 2, 500, 300)))
    bench_high = np.where(urban_class == 4, 2500,
                 np.where(urban_class == 3, 1500,
                 np.where(urban_class == 2, 1000, 800)))
    
    # Rooftop premium
    bench_low = bench_low + is_rooftop * 300
    bench_high = bench_high + is_rooftop * 300
    
    # 30% tolerance
    within = (predicted_fmv >= bench_low * 0.7) & (predicted_fmv <= bench_high * 1.3)
    above = predicted_fmv > bench_high * 1.3
    below = predicted_fmv < bench_low * 0.7
    
    return {
        'compliance': np.mean(within),
        'above_pct': np.mean(above),
        'below_pct': np.mean(below),
    }


def defense_layer_4_land_value(df, predicted_fmv):
    """Layer 4: County/state land value correlation."""
    state_pred = pd.DataFrame({'state': df['state'].values,
                                'pred': predicted_fmv,
                                'land': df['underlying_land_value_psf'].values})
    agg = state_pred.groupby('state').agg(
        pred_median=('pred', 'median'),
        land_median=('land', 'median'),
        count=('pred', 'count')
    ).query('count >= 5')
    
    if len(agg) < 3:
        return {'pearson_r': np.nan, 'spearman_rho': np.nan}
    
    pr, _ = pearsonr(agg['pred_median'], agg['land_median'])
    sr, _ = spearmanr(agg['pred_median'], agg['land_median'])
    return {'pearson_r': pr, 'spearman_rho': sr}


def defense_layer_5_expert(df, predicted_fmv, model_class):
    """Layer 5: Simulated domain expert validation."""
    # Simulate expert estimates = true_fmv + 12% noise
    expert_fmv = df['true_fmv'].values * np.random.lognormal(0, 0.12, len(df))
    
    # Expert classification
    expert_dev = (df['current_monthly_rent'].values - expert_fmv) / expert_fmv
    expert_class = np.where(expert_dev > 0.15, 'OVERPAYING',
                   np.where(expert_dev < -0.15, 'UNDERPAYING', 'AT_MARKET'))
    
    # Agreement between model and expert
    mape = np.mean(np.abs(predicted_fmv - expert_fmv) / expert_fmv)
    kappa = cohen_kappa_score(model_class, expert_class)
    return {'expert_mape': mape, 'expert_kappa': kappa}

## 4. Define Stress Scenarios

In [ ]:
def generate_scenario(df_raw, scenario_name):
    """Generate a stress-tested version of the data."""
    df = df_raw.copy()
    N = len(df)
    
    if scenario_name == 'A_baseline':
        pass  # use data as-is
    
    elif scenario_name == 'B_symmetric_20pct':
        # Re-generate rent with 20% noise
        noise = np.random.lognormal(0, 0.20, N)
        base = df['true_fmv'].values / (1.02 ** df['lease_age_years'].values)
        df['current_monthly_rent'] = np.clip(base * noise * df['cumulative_escalation_factor'].values, 100, 15000)
        dev = (df['current_monthly_rent'].values - df['true_fmv'].values) / df['true_fmv'].values
        df['true_class'] = np.where(dev > 0.15, 'OVERPAYING',
                           np.where(dev < -0.15, 'UNDERPAYING', 'AT_MARKET'))
    
    elif scenario_name == 'C_symmetric_50pct':
        noise = np.random.lognormal(0, 0.50, N)
        base = df['true_fmv'].values / (1.02 ** df['lease_age_years'].values)
        df['current_monthly_rent'] = np.clip(base * noise * df['cumulative_escalation_factor'].values, 100, 15000)
        dev = (df['current_monthly_rent'].values - df['true_fmv'].values) / df['true_fmv'].values
        df['true_class'] = np.where(dev > 0.15, 'OVERPAYING',
                           np.where(dev < -0.15, 'UNDERPAYING', 'AT_MARKET'))
    
    elif scenario_name == 'D_asymmetric_20pct':
        rent = df['true_fmv'].values.copy()
        roles = np.random.choice(['over', 'under', 'at'], N, p=[0.70, 0.15, 0.15])
        rent[roles == 'over']  *= (1 + np.random.uniform(0.05, 0.40, (roles == 'over').sum()))
        rent[roles == 'under'] *= (1 - np.random.uniform(0.05, 0.15, (roles == 'under').sum()))
        rent[roles == 'at']    *= np.random.uniform(0.97, 1.03, (roles == 'at').sum())
        df['current_monthly_rent'] = np.clip(rent, 100, 15000)
        dev = (df['current_monthly_rent'].values - df['true_fmv'].values) / df['true_fmv'].values
        df['true_class'] = np.where(dev > 0.15, 'OVERPAYING',
                           np.where(dev < -0.15, 'UNDERPAYING', 'AT_MARKET'))
    
    elif scenario_name == 'E_asymmetric_40pct':
        rent = df['true_fmv'].values.copy()
        roles = np.random.choice(['over', 'under', 'at'], N, p=[0.70, 0.15, 0.15])
        rent[roles == 'over']  *= (1 + np.random.uniform(0.15, 0.70, (roles == 'over').sum()))
        rent[roles == 'under'] *= (1 - np.random.uniform(0.05, 0.15, (roles == 'under').sum()))
        rent[roles == 'at']    *= np.random.uniform(0.97, 1.03, (roles == 'at').sum())
        df['current_monthly_rent'] = np.clip(rent, 100, 15000)
        dev = (df['current_monthly_rent'].values - df['true_fmv'].values) / df['true_fmv'].values
        df['true_class'] = np.where(dev > 0.15, 'OVERPAYING',
                           np.where(dev < -0.15, 'UNDERPAYING', 'AT_MARKET'))
    
    elif scenario_name == 'F_missing_features':
        # Replace key features with noise
        df['census_median_home_value'] = np.random.permutation(df['census_median_home_value'].values)
        df['scarcity_index'] = np.random.permutation(df['scarcity_index'].values)
    
    return df

SCENARIOS = ['A_baseline', 'B_symmetric_20pct', 'C_symmetric_50pct',
             'D_asymmetric_20pct', 'E_asymmetric_40pct', 'F_missing_features']
print(f'Defined {len(SCENARIOS)} stress scenarios.')

## 5. Run All Scenarios + All Defense Layers

In [ ]:
all_results = {}

for scenario in SCENARIOS:
    print(f'\n{"="*60}')
    print(f'SCENARIO: {scenario}')
    print(f'{"="*60}')
    
    # Generate scenario data
    np.random.seed(42)
    df_scenario = generate_scenario(df_raw, scenario)
    
    # Encode categoricals
    df_scen_enc, cat_cols = encode_categoricals(df_scenario)
    x_mkt_enc = X_MARKET + cat_cols
    all_ft_enc = ALL_FEATURES + cat_cols
    
    # Run pipeline
    print('  Running pipeline...')
    pipe = run_full_pipeline(df_scen_enc, x_mkt_enc, all_ft_enc)
    print(f'  MAPE: {pipe["mape"]*100:.1f}%, R2: {pipe["r2"]:.3f}, '
          f'Class Acc: {pipe["class_accuracy"]*100:.1f}%')
    
    # Run all defense layers
    print('  Running defense layers...')
    d1 = defense_layer_1_isolation_forest(df_scenario, pipe['predicted_fmv'], pipe['model_class'])
    d2 = defense_layer_2_lof(df_scenario, pipe['predicted_fmv'], pipe['model_class'])
    d3 = defense_layer_3_benchmarks(df_scenario, pipe['predicted_fmv'])
    d4 = defense_layer_4_land_value(df_scenario, pipe['predicted_fmv'])
    d5 = defense_layer_5_expert(df_scenario, pipe['predicted_fmv'], pipe['model_class'])
    
    # Three-way agreement (model + IF + LOF)
    three_way = np.mean(
        (pipe['model_class'] == d1['if_class']) & 
        (pipe['model_class'] == d2['lof_class'])
    )
    
    print(f'  L1-IF Kappa: {d1["kappa"]:.3f}, L2-LOF Kappa: {d2["kappa"]:.3f}')
    print(f'  L3-Benchmark Compliance: {d3["compliance"]*100:.1f}%')
    print(f'  L4-Land Pearson r: {d4["pearson_r"]:.3f}')
    print(f'  L5-Expert Kappa: {d5["expert_kappa"]:.3f}')
    print(f'  Three-way agreement: {three_way*100:.1f}%')
    
    all_results[scenario] = {
        'mape': pipe['mape'],
        'r2': pipe['r2'],
        'class_accuracy': pipe['class_accuracy'],
        'bias': pipe['bias'],
        'L1_IF_kappa': d1['kappa'],
        'L1_IF_agreement': d1['agreement'],
        'L2_LOF_kappa': d2['kappa'],
        'L2_LOF_agreement': d2['agreement'],
        'L3_compliance': d3['compliance'],
        'L3_above_pct': d3['above_pct'],
        'L4_pearson_r': d4['pearson_r'],
        'L4_spearman_rho': d4['spearman_rho'],
        'L5_expert_kappa': d5['expert_kappa'],
        'L5_expert_mape': d5['expert_mape'],
        'three_way_agreement': three_way,
    }

print(f'\nAll {len(SCENARIOS)} scenarios complete.')

## 6. Master Scorecard

In [ ]:
# Build summary dataframe
scorecard = pd.DataFrame(all_results).T
scorecard.index.name = 'Scenario'

# Format for display
display_cols = {
    'mape': 'MAPE',
    'r2': 'R-squared',
    'class_accuracy': 'Class Acc',
    'bias': 'Bias ($/mo)',
    'L1_IF_kappa': 'L1: IF Kappa',
    'L2_LOF_kappa': 'L2: LOF Kappa',
    'L3_compliance': 'L3: Bench Comply',
    'L4_pearson_r': 'L4: Land r',
    'L5_expert_kappa': 'L5: Expert Kappa',
    'three_way_agreement': '3-Way Agree',
}
scorecard_disp = scorecard[list(display_cols.keys())].rename(columns=display_cols)

# Format percentages
for col in ['MAPE', 'Class Acc', 'L3: Bench Comply', '3-Way Agree']:
    scorecard_disp[col] = (scorecard_disp[col] * 100).round(1).astype(str) + '%'
for col in ['R-squared', 'L1: IF Kappa', 'L2: LOF Kappa', 'L4: Land r', 'L5: Expert Kappa']:
    scorecard_disp[col] = scorecard_disp[col].round(3)
scorecard_disp['Bias ($/mo)'] = scorecard_disp['Bias ($/mo)'].round(0)

print('='*100)
print('DEFENSE LAYER SCORECARD -- ALL SCENARIOS')
print('='*100)
print(scorecard_disp.to_string())

## 7. Detection Classification (TP / TN / FP / FN)

In [ ]:
# Define which scenarios have real problems vs clean
problem_scenarios = {'B_symmetric_20pct', 'C_symmetric_50pct',
                     'D_asymmetric_20pct', 'E_asymmetric_40pct', 'F_missing_features'}
clean_scenarios = {'A_baseline'}

# Thresholds for "layer detects a problem"
THRESHOLDS = {
    'L1_IF_kappa': 0.30,       # Kappa below this = problem detected
    'L2_LOF_kappa': 0.30,
    'L3_compliance': 0.50,     # Compliance below 50% = problem detected
    'L4_pearson_r': 0.30,      # Correlation below this = problem detected
    'L5_expert_kappa': 0.30,
}

detection_matrix = []
for scenario in SCENARIOS:
    has_problem = scenario in problem_scenarios
    row = {'Scenario': scenario, 'Has_Problem': has_problem}
    
    for layer, thresh in THRESHOLDS.items():
        value = all_results[scenario][layer]
        layer_flags_problem = value < thresh
        
        if has_problem and layer_flags_problem:
            status = 'TP'  # Correctly detected
        elif has_problem and not layer_flags_problem:
            status = 'FN'  # Missed the problem
        elif not has_problem and layer_flags_problem:
            status = 'FP'  # False alarm
        else:
            status = 'TN'  # Correctly said no problem
        
        row[layer] = status
    
    detection_matrix.append(row)

det_df = pd.DataFrame(detection_matrix).set_index('Scenario')
print('DETECTION MATRIX (TP=caught problem, FN=MISSED, TN=correct clear, FP=false alarm)')
print(det_df.to_string())

In [ ]:
# Per-layer reliability
layer_names = list(THRESHOLDS.keys())
layer_reliability = {}
for layer in layer_names:
    counts = det_df[layer].value_counts()
    tp = counts.get('TP', 0)
    tn = counts.get('TN', 0)
    fp = counts.get('FP', 0)
    fn = counts.get('FN', 0)
    reliability = (tp + tn) / (tp + tn + fp + fn)
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 1.0
    layer_reliability[layer] = {
        'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn,
        'Reliability': reliability,
        'Detection_Rate': detection_rate,
    }

rel_df = pd.DataFrame(layer_reliability).T
rel_df.index = ['L1: Isolation Forest', 'L2: LOF', 'L3: Benchmarks',
                'L4: Land Value', 'L5: Expert']
print('\nPER-LAYER RELIABILITY')
print(rel_df.to_string())

## 8. Visualisations

In [ ]:
# --- Heatmap of detection matrix ---
status_map = {'TP': 2, 'TN': 1, 'FP': -1, 'FN': -2}
det_numeric = det_df[layer_names].replace(status_map).astype(float)

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Heatmap
cmap = sns.diverging_palette(10, 130, n=5, as_cmap=True)
sns.heatmap(det_numeric, annot=det_df[layer_names].values, fmt='',
            cmap=cmap, center=0, ax=axes[0],
            xticklabels=['L1:IF', 'L2:LOF', 'L3:Bench', 'L4:Land', 'L5:Expert'],
            yticklabels=[s.replace('_', ' ') for s in SCENARIOS])
axes[0].set_title('Detection Matrix\n(TP=green, FN=red, TN=light, FP=light red)')

# Bar chart of reliability
bars = axes[1].barh(rel_df.index, rel_df['Reliability'], color='steelblue', edgecolor='white')
axes[1].set_xlim(0, 1)
axes[1].set_xlabel('Reliability (TP+TN rate)')
axes[1].set_title('Defense Layer Reliability')
for bar, val in zip(bars, rel_df['Reliability']):
    axes[1].text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,
                f'{val:.1%}', va='center', fontsize=10)

# MAPE across scenarios
mape_vals = [all_results[s]['mape'] * 100 for s in SCENARIOS]
colors_bar = ['green' if m < 15 else 'orange' if m < 20 else 'red' for m in mape_vals]
axes[2].barh([s.replace('_', '\n') for s in SCENARIOS], mape_vals, color=colors_bar, edgecolor='white')
axes[2].axvline(15, color='green', ls='--', label='Target (15%)')
axes[2].axvline(20, color='red', ls='--', label='Threshold (20%)')
axes[2].set_xlabel('MAPE (%)')
axes[2].set_title('Model MAPE by Scenario')
axes[2].legend(fontsize=8)

plt.suptitle('Simulation 07 -- Defense Layer Scorecard', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# --- Spider / Radar chart of detection rates ---
from matplotlib.patches import FancyBboxPatch

categories = ['L1: IF', 'L2: LOF', 'L3: Bench', 'L4: Land', 'L5: Expert']
values = rel_df['Detection_Rate'].values.tolist()
values += values[:1]  # close the polygon
angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]

fig, ax = plt.subplots(figsize=(7, 7), subplot_kw=dict(polar=True))
ax.fill(angles, values, alpha=0.25, color='steelblue')
ax.plot(angles, values, 'o-', color='steelblue', linewidth=2)
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)
ax.set_ylim(0, 1)
ax.set_title('Defense Layer Detection Rate\n(higher = catches more problems)', fontsize=13, pad=20)
plt.tight_layout()
plt.show()

## 9. GO / NO-GO Decision

In [ ]:
baseline = all_results['A_baseline']
asym20 = all_results['D_asymmetric_20pct']

# Criteria
c1 = baseline['mape'] < 0.15
c2 = baseline['class_accuracy'] > 0.70

# Defense layers detect 20% asymmetric bias with >= 3/5 layers
det_row = det_df.loc['D_asymmetric_20pct', layer_names]
layers_detected = (det_row == 'TP').sum()
c3 = layers_detected >= 3

all_pass = c1 and c2 and c3

print('='*60)
print('GO / NO-GO DECISION')
print('='*60)
print(f'''
Criterion 1: Baseline MAPE < 15%
  Result: {baseline["mape"]*100:.1f}%  --  {"PASS" if c1 else "FAIL"}

Criterion 2: Baseline Classification Accuracy > 70%
  Result: {baseline["class_accuracy"]*100:.1f}%  --  {"PASS" if c2 else "FAIL"}

Criterion 3: Defense layers detect 20% asymmetric bias (>=3/5 layers)
  Result: {layers_detected}/5 layers detected  --  {"PASS" if c3 else "FAIL"}

{'='*60}
DECISION:  {">>> GO -- PROCEED TO REAL DATA <<<" if all_pass else ">>> NO-GO -- FIX ISSUES FIRST <<<"}
{'='*60}
''')

# Summary of most/least reliable layers
print('LAYER RECOMMENDATIONS:')
print(f'  Most reliable automated: {rel_df["Reliability"].idxmax()} ({rel_df["Reliability"].max():.0%})')
print(f'  Least reliable:          {rel_df["Reliability"].idxmin()} ({rel_df["Reliability"].min():.0%})')
print(f'\n  Keep all layers -- redundancy is the point.')
print(f'  Expert validation (L5) remains the ultimate check.')